In [2]:
import pandas as pd
import numpy as np
import requests
import json

import matplotlib.pyplot as plt
from scipy.stats import linregress

from pathlib import Path

In [3]:
CA_cases_per_100_path = Path("Datasets/CA_adults_with_diabetes_per_100.csv")
CA_cases_per_100_data = pd.read_csv(CA_cases_per_100_path)

In [8]:
CA_cases_per_100_data.head()

,Geography,Year,Strata,Strata Name,Percent,Lower 95% CL,Upper 95% CL,Standard Error
0,California,2018,Total population,Total population,10.4,8.9,11.9,0.8
1,California,2018,Race-Ethnicity,White,8.4,6.9,9.9,0.8
2,California,2018,Race-Ethnicity,African-American,12.3,6.0,18.6,3.2
3,California,2018,Race-Ethnicity,Asian,8.5,3.2,13.9,2.7
4,California,2018,Race-Ethnicity,Hispanic,12.1,9.0,15.1,1.6


In [6]:
#just cause
data_count = len(CA_cases_per_100_data)
data_count

147

In [12]:
#group by year
year = CA_cases_per_100_data.groupby("Year")
year.groups

{2012: [126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146], 2013: [105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125], 2014: [84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104], 2015: [63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83], 2016: [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62], 2017: [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], 2018: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}

In [18]:
reduced_df = CA_cases_per_100_data[["Year","Strata Name","Percent"]]
reduced_df

,Year,Strata Name,Percent
0,2018,Total population,10.4
1,2018,White,8.4
2,2018,African-American,12.3
3,2018,Asian,8.5
4,2018,Hispanic,12.1
...,...,...,...
142,2012,"$25,000 to $34,999",11.9
143,2012,"$35,000 to $49,999",9.5
144,2012,"$50,000 and above",6.3
145,2012,Male,9.7


In [23]:
total_pop = reduced_df.loc[reduced_df["Strata Name"] =="Total population"]
total_pop

,Year,Strata Name,Percent
0,2018,Total population,10.4
21,2017,Total population,8.2
42,2016,Total population,10.2
63,2015,Total population,9.0
84,2014,Total population,9.4
105,2013,Total population,9.5
126,2012,Total population,9.2


In [ ]:
#create line plot but the lines are connected
#create bar plot